# **Deploy Gemma 3-4B-Instruct and Test before Fine-tuning for Receipt Extraction**

In [ ]:
!pip install datasets transformers accelerate peft trl bitsandbytes python-dotenv packaging

In [ ]:
# Import library after install all libraries
import os
import torch
import json
from transformers import pipeline
from peft import LoraConfig

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
# Load environment and save HuggingFace token
from dotenv import load_dotenv
load_dotenv("env.txt")
token = os.getenv("HUGGINGFACE_TOKEN")
!huggingface-cli login --token {token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `maujadiaiengineer` has been saved to /home/ec2-user/.cache/huggingface/stored_tokens
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful.
The current active token is: `maujadiaiengineer`


In [ ]:
# Create inference pipeline based my Gemma fine-tuned model
pipeline = pipeline(
    task="image-text-to-text", # required for image input
    model="google/gemma-3-4b-it",
    device="cuda",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


In [ ]:
# Create function of receipt extraction
def receipt_extraction(image):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": "Extract store name, date and total information based this receipt photo"},
            ]
        },
    ]

    response = pipeline(text=messages, max_new_tokens=128, return_full_text=False, temperature=1)
    result = response[0]['generated_text']
    return result

In [ ]:
receipt_extraction("gokbbq.jpg")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0629 08:35:09.729000 14632 site-packages/torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


"Here's the extracted information from the receipt photo:\n\n*   **Store Name:** GO! K BBQ\n*   **Address:** 76 Amoy Street, Singapore 069895\n*   **Date:** 24/Dec/2023\n*   **Time:** 08:01:07 PM\n*   **Total:** $92.66"

In [ ]:
receipt_extraction("ovenfc.jpg")

"Here's the information extracted from the receipt image:\n\n*   **Store Name:** Oven & Fried Chicken\n*   **Address:** 230 Tanjong Katong Road, Singapore 437018\n*   **Date:** 09/Feb/2024 02:53:02 PM\n*   **Total:** 51.55"

In [ ]:
receipt_extraction("strongflour.jpg")

'Here’s the extracted information from the receipt image:\n\n*   **Store Name:** STRONG FLOUR\n*   **Date:** 27-05-18\n*   **Time:** 13:57\n*   **Total:** 77.05'